In [12]:
# Bulk Downloader
import pandas as pd
nifty50 = pd.read_csv("data/ind_nifty50list.csv")
niftynext50 = pd.read_csv("data/ind_niftynext50list.csv")
midcap50 = pd.read_csv("data/ind_niftymidcap50list.csv")

portfolio = midcap50.Symbol
industry = niftynext50.Industry.unique()

In [24]:
#portfolio = ["TCS","INFY","RIIL","SBIN","DRREDDY","AIRTEL","HDFC","ADANIPORTS","KOTAKBANK"]
for sym in portfolio:
    symbol = sym
    print("Downloading data for "+sym)
    %run "freedom_nse_downloader.ipynb"

In [3]:
from datetime import date

max_holding = 100

source = "CSV"

symbol = 'HDFC'

myDate="31-Mar-2019"

if myDate != "":
    print("Downloading data")
    filename ="./data/"+symbol+"_"+date.today().strftime("%d_%b_%Y")+".csv"
else:
    filename ="./data/"+symbol+"_"+myDate+".csv"

#dateTimeFormat = '%Y-%m-%d'
dateTimeFormat = '%d-%b-%Y'

# filter
days = 30 * 24

# frequency
freq = "1D"

%run "freedom_indicators_calculation.ipynb"


rangeMin = price.index[-150]
rangeMax = price.index[-1]

########## Algorithm #############
from enum import Enum
class S(Enum):
    BUY = 1
    SELL = 2
    HOLD = 3
    
price.dropna(inplace=True)

labels = pd.DataFrame(data = {"holdings":np.zeros(price.index.shape[0])},index=price.index)

#Entry Criterion
labels[
    ((price.bbup - price.close)/(price.bbup - price.bbmid) > 0.15)&(
    ((price.slowk<=10) & (price.close<price.bblow) &(price.RSI<30) ) |
    ((price.slowk<=17) & (price.aroonup>price.aroondown)) |
    (( price.bbup-price.close > 0.1 * (price.bbup - price.bbmid))&(price.macd.shift(3)<0)&(price.macd.shift(2)<0)&(price.macd.shift(1)<0)&(price.macd>=1) &(price.RSI<70)&(price.aroonup>price.aroondown))|
#   ((price.slowk < 15) & ( price.close< price.bbmid - (price.bbmid - price.bblow)*0.2))|
    (((price.aroonup>price.aroondown) | (price.slowk<17)) & (price.close < price.bbmid) & (price['macdhist'] > price['macdhist'].shift(1)) & (price['macdhist'].shift(1) < 0 ) & (price['macdhist'] >= 0 ))
    )
] = S.BUY

#Exit Criterion
labels[
     (((price.slowk.shift(3)>=80) | (price.slowk.shift(2)>=80) | (price.slowk.shift(1)>=80)) & (price.slowk <75) &(price.aroondown>price.aroonup) & (price.RSI>=60) )|
     ((price.bbup - price.close <=-5 ))| 
     ((price.close > price.bbmid + (price.bbup - price.bbmid) * 0.5 ) & (price['macdhist'] < price['macdhist'].shift(1)) & (price['macdhist'].shift(1) > 0 ) & (price['macdhist'] <= 0 ))] = S.SELL


# buy and sell decisions are stored in arrays for plotting purpose(visualisation)

buy = price[labels['holdings']==S.BUY]
sell = price[labels['holdings']==S.SELL]
#################################

%run "freedom_plot.ipynb"


freedom_indicators_calculation.ipynb:1: FutureWarning:

using a dict with renaming is deprecated and will be removed in a future version



This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]
[ (6,1) x1,y6 ]
[ (7,1) x1,y7 ]
[ (8,1) x1,y8 ]



## Entry Criterion
- MACD crossover when market is trending up
- BB when market is moving sideways

## Exit Criterion
- Stochastics

## Identify market trend
- low ATR means market moving sideways
- Flat(gradient=0) MACD means market is moving sideways
- low ATR indicates low volatility - market is moving sideways
- High ATR indicates high volatility - market is trending
- Gradient of current - last 15 days